In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from spectradb.main import Database
from spectradb.dataloaders import FTIRDataLoader
from pathlib import Path
import pandas as pd
import sqlite3

In [3]:
path = Path().resolve().parents[0]
data_path = path/"Only Dataloaders"/"Example.SPA"

In [4]:
ftir = FTIRDataLoader(data_path)

Data generated from FTIR spectrometer
File: Example


In [5]:
with Database("db.sqlite") as db: 
    db.add_sample(ftir)

In [6]:
conn = sqlite3.connect("db.sqlite")
pd.read_sql("SELECT * FROM measurements", conn)

,measurement_id,instrument_id,measurement_date,sample_name,internal_code,collected_by,comments,data,signal_metadata
0,1,FTIR,2024-03-24,NA,NA,NA,NA,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,..."
1,2,FTIR,2024-03-24,NA,NA,NA,NA,"[3.7390708923339844, 3.7333016395568848, 3.686...","{""Wavenumbers"": [3999, 3997, 3995, 3993, 3991,..."
